In [1]:
# Initial Imports
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
from datetime import datetime

import requests
import json

load_dotenv()

True

In [2]:
# Retrieving AlphaVantage API Key
alphavantage_api_key = os.getenv("ALPHAVANTAGE_API_KEY")

# Setting AlphaVantage base URL
url_alpha = "https://www.alphavantage.co/query?"

In [3]:
def getIncS(ticker):
    """
    Function for calling AlphaVantage for income statement. Returns a dataframe with the various financials in the quarterly reports
    """
    params = {
        "function":"INCOME_STATEMENT",
        "symbol":ticker,
        "apikey":alphavantage_api_key
    }
    
    response_data = requests.get(url_alpha, params = params).json()
    
    qr_df = pd.DataFrame(response_data["quarterlyReports"])
    qr_df.set_index(qr_df["fiscalDateEnding"], inplace = True)
    qr_df.drop(columns = "fiscalDateEnding", inplace = True)
    
    return qr_df

In [4]:
def getBalS(ticker):
    """
    Function for calling AlphaVantage for balance sheet. Returns a dataframe with the various financials in the quarterly reports
    """
    params = {
        "function":"BALANCE_SHEET",
        "symbol":ticker,
        "apikey":alphavantage_api_key
    }
    
    response_data = requests.get(url_alpha, params = params).json()
    
    qr_df = pd.DataFrame(response_data["quarterlyReports"])
    qr_df.set_index(qr_df["fiscalDateEnding"], inplace = True)
    qr_df.drop(columns = "fiscalDateEnding", inplace = True)
    
    return qr_df

In [5]:
def getSCF(ticker):
    """
    Function for calling AlphaVantage for statement of cash flows. Returns a dataframe with the various financials in the quarterly reports
    """
    params = {
        "function":"BALANCE_SHEET",
        "symbol":ticker,
        "apikey":alphavantage_api_key
    }
    
    response_data = requests.get(url_alpha, params = params).json()
    
    qr_df = pd.DataFrame(response_data["quarterlyReports"])
    qr_df.set_index(qr_df["fiscalDateEnding"], inplace = True)
    qr_df.drop(columns = "fiscalDateEnding", inplace = True)
    
    return qr_df

In [6]:
def getEPS(ticker):
    """
    Function for calling AlphaVantage for earnings (EPS). Returns a dataframe with the various financials in the quarterly reports
    """
    params = {
        "function":"EARNINGS",
        "symbol":ticker,
        "apikey":alphavantage_api_key
    }
    
    response_data = requests.get(url_alpha, params = params).json()
    
    qr_df = pd.DataFrame(response_data["quarterlyEarnings"])
    qr_df.set_index(qr_df["fiscalDateEnding"], inplace = True)
    qr_df.drop(columns = "fiscalDateEnding", inplace = True)
    
    return qr_df

In [7]:
def calcFRatios(ticker):
    """
    Function that calculates the following financial ratios based on the quarterly financial statements of the company:
    - Working capital Ratio
    - Acid-Test Ratio (Quick Ratio)
    - EPS
    - Debt-Equity Ratio
    
    Ratios are returned in a dataframe
    """
    
    # Retrieving Balance Sheet and Earnings
    balS = getBalS(ticker)
    eps = getEPS(ticker)
    
    # Creating return DataFrame
    columns = ["workingCapitalRatio", "acidTestRatio", "eps", "debtEquityRatio", "currentAssets", "currentLiabilities", "inventory", "longTermDebt", "shortTermDebt", "shareholderEquity"]
    ratio_df = pd.DataFrame(index = balS.index, columns = columns)
    
    # Filling out DataFrame with provided data
    ratio_df["currentAssets"] = balS["totalCurrentAssets"].str.replace("None","0").astype("float")
    ratio_df["currentLiabilities"] = balS["totalCurrentLiabilities"].str.replace("None","0").astype("float")
    ratio_df["inventory"] = balS["inventory"].str.replace("None","0").astype("float")
    ratio_df["longTermDebt"] = balS["longTermDebt"].str.replace("None","0").astype("float")
    ratio_df["shortTermDebt"] = balS["shortTermDebt"].str.replace("None","0").astype("float")
    ratio_df["shareholderEquity"] = balS["totalShareholderEquity"].str.replace("None","0").astype("float")
    
    # Calculating the ratios
    
    ratio_df["workingCapitalRatio"] = ratio_df["currentAssets"] / ratio_df["currentLiabilities"]
    ratio_df["acidTestRatio"] = (ratio_df["currentAssets"] - ratio_df["inventory"]) / ratio_df["currentLiabilities"]
    ratio_df["eps"] = eps["reportedEPS"]
    ratio_df["debtEquityRatio"] = (ratio_df["longTermDebt"] + ratio_df["shortTermDebt"]) / ratio_df["shareholderEquity"]
    
    return ratio_df